# Covid 19 - predictions for death cases

### Loading libraries

In [13]:
import pandas as pd
import pandas as pd 
import random

In [14]:
import math
import time
import datetime
import operator 
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [15]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [16]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
plt.style.use('seaborn-dark')
#print(plt.style.available)
%matplotlib inline

from sklearn import metrics

In [17]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV

In [18]:
import warnings
warnings.filterwarnings("ignore")

### Preparation for predictions

#### Data Loading

In [19]:
#dataset
covid_df= pd.read_csv("corona_pred_china.csv")
covid_df.head()

,ObservationDate,Country,Continent,Confirmed,Deaths,Recovered,Active Cases,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,GDP ($ per capita)
0,2020-01-22,Hong Kong,Asia,0.0,0.0,0.0,0.0,6940432.0,1092.0,"6355,7","5,24",28800.0
1,2020-01-22,Japan,Asia,2.0,0.0,0.0,2.0,127463611.0,377835.0,"337,4",0,28200.0
2,2020-01-22,Macao,Asia,1.0,0.0,0.0,1.0,453125.0,28.0,"16183,0","4,86",19400.0
3,2020-01-22,South Korea,Asia,1.0,0.0,0.0,1.0,48846823.0,98480.0,"496,0",0,17800.0
4,2020-01-22,Taiwan,Asia,1.0,0.0,0.0,1.0,23036087.0,35980.0,"640,3",0,23400.0


#### Data transformations

In [20]:
#Converting "Observation Date" into Datetime format
covid_df["ObservationDate"]=pd.to_datetime(covid_df["ObservationDate"])
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3811 entries, 0 to 3810
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ObservationDate             3811 non-null   datetime64[ns]
 1   Country                     3811 non-null   object        
 2   Continent                   3811 non-null   object        
 3   Confirmed                   3811 non-null   float64       
 4   Deaths                      3811 non-null   float64       
 5   Recovered                   3811 non-null   float64       
 6   Active Cases                3811 non-null   float64       
 7   Population                  3811 non-null   float64       
 8   Area (sq. mi.)              3811 non-null   float64       
 9   Pop. Density (per sq. mi.)  3811 non-null   object        
 10  Net migration               3811 non-null   object        
 11  GDP ($ per capita)          3811 non-null   float64     

In [21]:
# Changing datatype of Population density and Net Migration columns as float
cols=["Pop. Density (per sq. mi.)","Net migration"]                             
for i in cols:
    covid_df[i]=covid_df[i].str.replace(",","").astype(float)
covid_df.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3811 entries, 0 to 3810
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ObservationDate             3811 non-null   datetime64[ns]
 1   Country                     3811 non-null   object        
 2   Continent                   3811 non-null   object        
 3   Confirmed                   3811 non-null   float64       
 4   Deaths                      3811 non-null   float64       
 5   Recovered                   3811 non-null   float64       
 6   Active Cases                3811 non-null   float64       
 7   Population                  3811 non-null   float64       
 8   Area (sq. mi.)              3811 non-null   float64       
 9   Pop. Density (per sq. mi.)  3811 non-null   float64       
 10  Net migration               3811 non-null   float64       
 11  GDP ($ per capita)          3811 non-null   float64     

In [22]:
covid_df.head()

,ObservationDate,Country,Continent,Confirmed,Deaths,Recovered,Active Cases,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,GDP ($ per capita)
0,2020-01-22,Hong Kong,Asia,0.0,0.0,0.0,0.0,6940432.0,1092.0,63557.0,524.0,28800.0
1,2020-01-22,Japan,Asia,2.0,0.0,0.0,2.0,127463611.0,377835.0,3374.0,0.0,28200.0
2,2020-01-22,Macao,Asia,1.0,0.0,0.0,1.0,453125.0,28.0,161830.0,486.0,19400.0
3,2020-01-22,South Korea,Asia,1.0,0.0,0.0,1.0,48846823.0,98480.0,4960.0,0.0,17800.0
4,2020-01-22,Taiwan,Asia,1.0,0.0,0.0,1.0,23036087.0,35980.0,6403.0,0.0,23400.0


In [23]:
# Sorting based on the country names
covid_df = covid_df.sort_values(["Country", "ObservationDate"]).reset_index(drop=True)

In [24]:
covid_df.shape

(3811, 12)

In [25]:
# Calculating days since the first occurance for each country
covid_df['days_since_first']=0
covid_df['previous_Deaths'] = 0
covid_df['death_rate'] = 0
previous_Deaths=0
confirm_difference=0

for i in covid_df.index[1:]:
    if covid_df['Country'][i]==covid_df['Country'][i-1]:
        covid_df.loc[i,'days_since_first']=(covid_df.at[i-1,'days_since_first']+1)
        previous_Deaths=covid_df.at[i-1,'Deaths']
        confirm_difference= covid_df.at[i,'Deaths']-previous_Deaths
        covid_df.loc[i,'previous_Deaths']=previous_Deaths        
        if ((previous_Deaths!=0) & (confirm_difference>0) ):
            covid_df.loc[i,'death_rate']=((confirm_difference/previous_Deaths))       
        else:
            covid_df.loc[i,'death_rate']=0
    else:
        covid_df.loc[i,'days_since_first'] = 0
        covid_df.loc[i,'previous_Deaths'] = 0
        covid_df.loc[i,'death_rate'] = 0  

In [26]:
# Ensuring that previous Deaths column type is int
covid_df['previous_Deaths'].astype(int)
covid_df=covid_df[covid_df['death_rate']>=0]

In [27]:
covid_df.set_index('Country',inplace=True)

#### Check for highest number of Deaths cases in Europe

In [66]:
covid_df_Europe = covid_df[covid_df['Continent'] == "Europe"].groupby('Country').agg(
                {
                     'Deaths':sum,    # Sum duration per group
                     'days_since_first': max,  # get the max of days
                }).sort_values(["Deaths", "days_since_first"]).tail()
covid_df_Europe.sort_values(["Deaths"], ascending=False)

,Deaths,days_since_first
Country,,
Italy,"33,959",51
Spain,"7,345",50
France,"2,859",58
United Kingdom,"1,100",51
Netherlands,690,24


In [89]:
covid_df.columns

Index(['ObservationDate', 'Continent', 'Confirmed', 'Deaths', 'Recovered',
       'Active Cases', 'Population', 'Area (sq. mi.)',
       'Pop. Density (per sq. mi.)', 'Net migration', 'GDP ($ per capita)',
       'days_since_first', 'previous_Confirmed', 'increase_rate'],
      dtype='object')

### Random Forest Regressor

In [31]:
X=covid_df.copy()
del X['Continent']
del X['ObservationDate']
del X['Confirmed']
del X['Recovered']
del X['Active Cases']
del X['GDP ($ per capita)']
#del X['previous_Activecases']
#del X['previous_Death']
#del X['previous_Recovered']



In [32]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3811 entries, Afghanistan to Zimbabwe
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Deaths                      3811 non-null   float64
 1   Population                  3811 non-null   float64
 2   Area (sq. mi.)              3811 non-null   float64
 3   Pop. Density (per sq. mi.)  3811 non-null   float64
 4   Net migration               3811 non-null   float64
 5   days_since_first            3811 non-null   int64  
 6   previous_Deaths             3811 non-null   float64
 7   death_rate                  3811 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 268.0+ KB


In [33]:
X["Net migration"].fillna(0, inplace=True)
X["Pop. Density (per sq. mi.)"].fillna(0, inplace=True)

In [34]:
X.isnull().any()

Deaths                        False
Population                    False
Area (sq. mi.)                False
Pop. Density (per sq. mi.)    False
Net migration                 False
days_since_first              False
previous_Deaths               False
death_rate                    False
dtype: bool

In [35]:
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,Deaths,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,days_since_first,previous_Deaths,death_rate
Deaths,1.00,0.01,-0.01,-0.02,0.00,0.16,1.00,0.07
Population,0.01,1.00,0.30,-0.07,-0.06,0.15,0.01,0.05
Area (sq. mi.),-0.01,0.30,1.00,-0.10,0.08,0.16,-0.01,0.04
Pop. Density (per sq. mi.),-0.02,-0.07,-0.10,1.00,0.29,0.14,-0.02,-0.04
Net migration,0.00,-0.06,0.08,0.29,1.00,0.13,0.00,0.01
days_since_first,0.16,0.15,0.16,0.14,0.13,1.00,0.16,0.19
previous_Deaths,1.00,0.01,-0.01,-0.02,0.00,0.16,1.00,0.06
death_rate,0.07,0.05,0.04,-0.04,0.01,0.19,0.06,1.00


In [36]:
# Pick only features that are correlated more than 0.2
y = X['Deaths']     

In [37]:
print(X.shape)
X.head()

(3811, 8)


,Deaths,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,days_since_first,previous_Deaths,death_rate
Country,,,,,,,,
Afghanistan,0.0,31056997.0,647500.0,480.0,2306.0,0,0.0,0.0
Afghanistan,0.0,31056997.0,647500.0,480.0,2306.0,1,0.0,0.0
Afghanistan,0.0,31056997.0,647500.0,480.0,2306.0,2,0.0,0.0
Afghanistan,0.0,31056997.0,647500.0,480.0,2306.0,3,0.0,0.0
Afghanistan,0.0,31056997.0,647500.0,480.0,2306.0,4,0.0,0.0


In [38]:
# To get rid of +e format
pd.options.display.float_format = '{:20,.0f}'.format

#### Train Test Split

In [39]:
X.drop(['Deaths'], axis=1, inplace=True)


# Break off validation set from training data

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, test_size=0.15, random_state=1)

In [40]:
print(f"X_train shape is {X_train.shape}")
print(f"X_test shape is {X_test.shape}")
# print(f"X_validate shape is {X_validate.shape}")
print(f"y_train is {y_train.shape}")
# print(f"y_validate shape is {y_validate.shape}")
print(f"y_test shape is {y_test.shape}")

X_train shape is (3239, 7)
X_test shape is (572, 7)
y_train is (3239,)
y_test shape is (572,)


## Random Forest

#### Defining the model and checking the best score

In [41]:
#DEFINE YOUR REGRESSOR and THE PARAMETERS GRID

regressor = RandomForestRegressor()
parameters = {"criterion": ['mse'], 
              "n_estimators":[10, 100, 400], 
              "min_samples_leaf": [0.1, 1, 5, 10, 20, 50], 
              "random_state" : [1]
            #  "max_features": "auto"
            # "oob_score ": TRUE
             }

In [42]:
#DEFINE YOUR GRIDSEARCH 
gs = GridSearchCV(regressor, parameters, cv=3) #with no params it reduces to a CV

gs = gs.fit(X_train.to_numpy(),y_train.to_numpy())

In [43]:
#summarize the results of your GRIDSEARCH
print('***GRIDSEARCH RESULTS***')
print("Best score: %f using %s" % (gs.best_score_, gs.best_params_))
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
params = gs.cv_results_['params']


***GRIDSEARCH RESULTS***
Best score: 0.970718 using {'criterion': 'mse', 'min_samples_leaf': 1, 'n_estimators': 10, 'random_state': 1}


In [44]:
columns = np.asarray(X.columns)
#columns = np.asarray([columns])
from sklearn import tree
r = tree.export_text(gs.best_estimator_.estimators_[0],feature_names=columns.tolist(), max_depth=3)
print(r)

|--- previous_Deaths <= 1015.50
|   |--- previous_Deaths <= 216.50
|   |   |--- previous_Deaths <= 66.50
|   |   |   |--- previous_Deaths <= 20.50
|   |   |   |   |--- truncated branch of depth 19
|   |   |   |--- previous_Deaths >  20.50
|   |   |   |   |--- truncated branch of depth 10
|   |   |--- previous_Deaths >  66.50
|   |   |   |--- previous_Deaths <= 112.50
|   |   |   |   |--- truncated branch of depth 6
|   |   |   |--- previous_Deaths >  112.50
|   |   |   |   |--- truncated branch of depth 6
|   |--- previous_Deaths >  216.50
|   |   |--- previous_Deaths <= 617.00
|   |   |   |--- previous_Deaths <= 506.00
|   |   |   |   |--- truncated branch of depth 5
|   |   |   |--- previous_Deaths >  506.00
|   |   |   |   |--- truncated branch of depth 2
|   |   |--- previous_Deaths >  617.00
|   |   |   |--- death_rate <= 0.43
|   |   |   |   |--- truncated branch of depth 4
|   |   |   |--- death_rate >  0.43
|   |   |   |   |--- value: [1266.00]
|--- previous_Deaths >  1015.50
|

####  Training the model

In [45]:
#we establish the model with the best estimator according to Gridsearch results
gs = gs.best_estimator_ 

gs = gs.fit(X_train.to_numpy(),y_train.to_numpy())

In [46]:
#Returns the coefficient of determination R^2 of the prediction.
#Explained variance score: 1 is perfect prediction
gs.score(X_test.to_numpy(), y_test.to_numpy())

0.9915568126875732

#### Testing the model

In [47]:
y_pred = gs.predict(X_test.to_numpy())

In [48]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred}).reset_index(drop=True)
df

,Actual,Predicted
0,3,3
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
567,0,0
568,4,4
569,0,0
570,0,0


In [49]:
# It is not giving correct results yet. Need to check
df=df[df["Actual"] != df["Predicted"]]
df

,Actual,Predicted
2,0,0
4,1,0
7,1,0
12,1,0
21,0,1
...,...,...
556,0,0
557,13,15
560,4,3
561,5,5


#### Evaluation results

In [50]:
print("MAE train: ", metrics.mean_absolute_error(y_train.to_numpy(), gs.predict(X_train.to_numpy()))) 
print("MSE train: ",metrics.mean_squared_error(y_train.to_numpy(), gs.predict(X_train.to_numpy())))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train.to_numpy(), gs.predict(X_train.to_numpy()))))
print("r2: ",np.sqrt(metrics.r2_score(y_train.to_numpy(), gs.predict(X_train.to_numpy()))))

print("MAE test: ", metrics.mean_absolute_error(y_test.to_numpy(), gs.predict(X_test.to_numpy()))) 
print("MSE test: ",metrics.mean_squared_error(y_test.to_numpy(), gs.predict(X_test.to_numpy())))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test.to_numpy(), gs.predict(X_test.to_numpy()))))
print("r2: ",np.sqrt(metrics.r2_score(y_test.to_numpy(), gs.predict(X_test.to_numpy()))))

MAE train:  1.0429453535041677
MSE train:  185.72987650509398
RMSE train:  13.628274891015883
r2:  0.9976827033703183
MAE test:  0.7363636363636364
MSE test:  35.2797202797203
RMSE test:  5.939673415240968
r2:  0.9957694575992845


## Random Forest Regression -  For different countries

### Using Kaggle dataset to cross check the results of future prediction 

In [51]:
covid_df_kaggle= pd.read_csv("covid_19_data_Kaggle.csv")

In [52]:
covid_df_kaggle["ObservationDate"]=pd.to_datetime(covid_df_kaggle["ObservationDate"])

In [53]:
covid_df_kaggle.shape

(13850, 8)

### Random Forest Regression - Italy

In [54]:
covid_df_kaggle_Italy = covid_df_kaggle[covid_df_kaggle["Country/Region"] == "Italy"].groupby("ObservationDate").agg(
                {
                     'Confirmed':sum,    # Sum duration per group
                     'Deaths':sum,    # Sum duration per group
                     'Recovered':sum    # Sum duration per group
                }).sort_values(["ObservationDate"]).reset_index()

In [55]:
covid_df_kaggle_Italy

,ObservationDate,Confirmed,Deaths,Recovered
0,2020-01-31,2,0,0
1,2020-02-01,2,0,0
2,2020-02-02,2,0,0
3,2020-02-03,2,0,0
4,2020-02-04,2,0,0
...,...,...,...,...
66,2020-04-06,"132,547","16,523","22,837"
67,2020-04-07,"135,586","17,127","24,392"
68,2020-04-08,"139,422","17,669","26,491"
69,2020-04-09,"143,626","18,279","28,470"


In [56]:
filter_condn = (covid_df_kaggle_Italy['ObservationDate'] > '2020-03-21') & (covid_df_kaggle_Italy['ObservationDate'] <= '2020-03-26')
covid_df_kaggle_Italy = covid_df_kaggle_Italy[filter_condn].reset_index()
covid_df_kaggle_Italy

X_Italy_df = X[X.index == "Italy"].copy()
X_Italy_test = X_Italy_df.head().copy()
X_Italy_test=X_Italy_test.reset_index(drop=True)
X_Italy_test['days_since_first'] = X_Italy_test['previous_Deaths'] = X_Italy_test['death_rate'] = 0
X_Italy_test.loc[0,'days_since_first'] =  X_Italy_df["days_since_first"].iloc[-1]+1
X_Italy_test.loc[0,'death_rate']=X_Italy_df["death_rate"].iloc[-1]+1
X_Italy_test.loc[0,'previous_Deaths'] = y[y.index == "Italy"].max()

for i in range(0,5):
    X_Italy_test.loc[i,'days_since_first'] = X_Italy_test.loc[0,'days_since_first'] + i

X_Italy_test

predictions=[]

for i in range(0,5):
    pred=int(gs.predict([X_Italy_test.iloc[i].to_numpy()]))
    X_Italy_test.loc[i+1,'previous_Deaths']=pred    
    predictions.append(pred)

pred = pd.DataFrame(predictions)

#### Prediction results
X_Italy_test['predicted_infections']=pred.iloc[:, 0] 
X_Italy_test=X_Italy_test[{'predicted_infections'}]
# Creating new column to have actual predictions and date from Kaggle dataset
X_Italy_test["Actual infections"] = covid_df_kaggle_Italy["Deaths"]
X_Italy_test["Date"] = covid_df_kaggle_Italy["ObservationDate"]
X_Italy_test = X_Italy_test.iloc[0:5]
X_Italy_test 

,predicted_infections,Actual infections,Date
0,"4,864","5,476",2020-03-22
1,"5,073","6,077",2020-03-23
2,"5,073","6,820",2020-03-24
3,"5,073","7,503",2020-03-25
4,"5,073","8,215",2020-03-26


### Random Forest Regression - France

In [57]:
covid_df_kaggle_France = covid_df_kaggle[covid_df_kaggle["Country/Region"] == "France"].groupby("ObservationDate").agg(
                {
                     'Confirmed':sum,    # Sum duration per group
                     'Deaths':sum,    # Sum duration per group
                     'Recovered':sum    # Sum duration per group
                }).sort_values(["ObservationDate"]).reset_index()

In [58]:
filter_condn = (covid_df_kaggle_France['ObservationDate'] > '2020-03-21') & (covid_df_kaggle_France['ObservationDate'] <= '2020-03-26')
covid_df_kaggle_France = covid_df_kaggle_France[filter_condn].reset_index()
covid_df_kaggle_France

X_France_df = X[X.index == "France"].copy()
X_France_test = X_France_df.head().copy()
X_France_test=X_France_test.reset_index(drop=True)
X_France_test['days_since_first'] = X_France_test['previous_Deaths'] = X_France_test['death_rate'] = 0
X_France_test.loc[0,'days_since_first'] =  X_France_df["days_since_first"].iloc[-1]+1
X_France_test.loc[0,'death_rate']=X_France_df["death_rate"].iloc[-1]+1
X_France_test.loc[0,'previous_Deaths'] = y[y.index == "France"].max()

for i in range(0,5):
    X_France_test.loc[i,'days_since_first'] = X_France_test.loc[0,'days_since_first'] + i

X_France_test

predictions=[]

for i in range(0,5):
    pred=int(gs.predict([X_France_test.iloc[i].to_numpy()]))
    X_France_test.loc[i+1,'previous_Deaths']=pred    
    predictions.append(pred)

pred = pd.DataFrame(predictions)

#### Prediction results
X_France_test['predicted_infections']=pred.iloc[:, 0] 
X_France_test=X_France_test[{'predicted_infections'}]
X_France_test["Actual infections"] = covid_df_kaggle_France["Deaths"]
X_France_test["Date"] = covid_df_kaggle_France["ObservationDate"]
X_France_test = X_France_test.iloc[0:5]
X_France_test 

,predicted_infections,Actual infections,Date
0,822,676,2020-03-22
1,883,862,2020-03-23
2,"1,028","1,102",2020-03-24
3,"1,288","1,333",2020-03-25
4,"1,506","1,698",2020-03-26


### Random Forest Regression - Spain

In [59]:
covid_df_kaggle_Spain = covid_df_kaggle[covid_df_kaggle["Country/Region"] == "Spain"].groupby("ObservationDate").agg(
                {
                     'Confirmed':sum,    # Sum duration per group
                     'Deaths':sum,    # Sum duration per group
                     'Recovered':sum    # Sum duration per group
                }).sort_values(["ObservationDate"]).reset_index()

In [60]:
filter_condn = (covid_df_kaggle_Spain['ObservationDate'] > '2020-03-21') & (covid_df_kaggle_Spain['ObservationDate'] <= '2020-03-26')
covid_df_kaggle_Spain = covid_df_kaggle_Spain[filter_condn].reset_index()
covid_df_kaggle_Spain

X_Spain_df = X[X.index == "Spain"].copy()
X_Spain_test = X_Spain_df.head().copy()
X_Spain_test=X_Spain_test.reset_index(drop=True)
X_Spain_test['days_since_first'] = X_Spain_test['previous_Deaths'] = X_Spain_test['death_rate'] = 0
X_Spain_test.loc[0,'days_since_first'] =  X_Spain_df["days_since_first"].iloc[-1]+1
X_Spain_test.loc[0,'death_rate']=X_Spain_df["death_rate"].iloc[-1]+1
X_Spain_test.loc[0,'previous_Deaths'] = y[y.index == "Spain"].max()

for i in range(0,5):
    X_Spain_test.loc[i,'days_since_first'] = X_Spain_test.loc[0,'days_since_first'] + i

X_Spain_test

predictions=[]

for i in range(0,5):
    pred=int(gs.predict([X_Spain_test.iloc[i].to_numpy()]))
    X_Spain_test.loc[i+1,'previous_Deaths']=pred    
    predictions.append(pred)

pred = pd.DataFrame(predictions)

#### Prediction results
X_Spain_test['predicted_infections']=pred.iloc[:, 0] 
X_Spain_test=X_Spain_test[{'predicted_infections'}]
X_Spain_test["Actual infections"] = covid_df_kaggle_Spain["Deaths"]
X_Spain_test["Date"] = covid_df_kaggle_Spain["ObservationDate"]
X_Spain_test = X_Spain_test .iloc[0:5]
X_Spain_test 

,predicted_infections,Actual infections,Date
0,"2,308","1,756",2020-03-22
1,"3,258","2,311",2020-03-23
2,"4,435","2,808",2020-03-24
3,"5,073","3,647",2020-03-25
4,"5,073","4,365",2020-03-26


### Random Forest Regression - Germany

In [62]:
covid_df_kaggle_Germany = covid_df_kaggle[covid_df_kaggle["Country/Region"] == "Germany"].groupby("ObservationDate").agg(
                {
                     'Confirmed':sum,    # Sum duration per group
                     'Deaths':sum,    # Sum duration per group
                     'Recovered':sum    # Sum duration per group
                }).sort_values(["ObservationDate"]).reset_index()

In [63]:
filter_condn = (covid_df_kaggle_Germany['ObservationDate'] > '2020-03-21') & (covid_df_kaggle_Germany['ObservationDate'] <= '2020-03-26')
covid_df_kaggle_Germany = covid_df_kaggle_Germany[filter_condn].reset_index()
covid_df_kaggle_Germany

X_Germany_df = X[X.index == "Germany"].copy()
X_Germany_test = X_Germany_df.head().copy()
X_Germany_test=X_Germany_test.reset_index(drop=True)
X_Germany_test['days_since_first'] = X_Germany_test['previous_Deaths'] = X_Germany_test['death_rate'] = 0
X_Germany_test.loc[0,'days_since_first'] =  X_Germany_df["days_since_first"].iloc[-1]+1
X_Germany_test.loc[0,'death_rate']=X_Germany_df["death_rate"].iloc[-1]+1
X_Germany_test.loc[0,'previous_Deaths'] = y[y.index == "Germany"].max()

for i in range(0,5):
    X_Germany_test.loc[i,'days_since_first'] = X_Germany_test.loc[0,'days_since_first'] + i

X_Germany_test

predictions=[]

for i in range(0,5):
    pred=int(gs.predict([X_Germany_test.iloc[i].to_numpy()]))
    X_Germany_test.loc[i+1,'previous_Deaths']=pred    
    predictions.append(pred)

pred = pd.DataFrame(predictions)

#### Prediction results
X_Germany_test['predicted_infections']=pred.iloc[:, 0] 
X_Germany_test=X_Germany_test[{'predicted_infections'}]
X_Germany_test["Actual infections"] = covid_df_kaggle_Germany["Deaths"]
X_Germany_test["Date"] = covid_df_kaggle_Germany["ObservationDate"]
X_Germany_test = X_Germany_test.iloc[0:5]
X_Germany_test 

,predicted_infections,Actual infections,Date
0,136,94,2020-03-22
1,175,123,2020-03-23
2,232,157,2020-03-24
3,316,206,2020-03-25
4,387,267,2020-03-26


### Random Forest Regression - Switzerland

In [64]:
covid_df_kaggle_Switzerland = covid_df_kaggle[covid_df_kaggle["Country/Region"] == "Switzerland"].groupby("ObservationDate").agg(
                {
                     'Confirmed':sum,    # Sum duration per group
                     'Deaths':sum,    # Sum duration per group
                     'Recovered':sum    # Sum duration per group
                }).sort_values(["ObservationDate"]).reset_index()

In [65]:
filter_condn = (covid_df_kaggle_Switzerland['ObservationDate'] > '2020-03-21') & (covid_df_kaggle_Switzerland['ObservationDate'] <= '2020-03-26')
covid_df_kaggle_Switzerland = covid_df_kaggle_Switzerland[filter_condn].reset_index()
covid_df_kaggle_Switzerland

X_Switzerland_df = X[X.index == "Switzerland"].copy()
X_Switzerland_test = X_Switzerland_df.head().copy()
X_Switzerland_test=X_Switzerland_test.reset_index(drop=True)
X_Switzerland_test['days_since_first'] = X_Switzerland_test['previous_Deaths'] = X_Switzerland_test['death_rate'] = 0
X_Switzerland_test.loc[0,'days_since_first'] =  X_Switzerland_df["days_since_first"].iloc[-1]+1
X_Switzerland_test.loc[0,'death_rate']=X_Switzerland_df["death_rate"].iloc[-1]+1
X_Switzerland_test.loc[0,'previous_Deaths'] = y[y.index == "Switzerland"].max()

for i in range(0,5):
    X_Switzerland_test.loc[i,'days_since_first'] = X_Switzerland_test.loc[0,'days_since_first'] + i

X_Switzerland_test

predictions=[]

for i in range(0,5):
    pred=int(gs.predict([X_Switzerland_test.iloc[i].to_numpy()]))
    X_Switzerland_test.loc[i+1,'previous_Deaths']=pred    
    predictions.append(pred)

pred = pd.DataFrame(predictions)

#### Prediction results
X_Switzerland_test['predicted_infections']=pred.iloc[:, 0] 
X_Switzerland_test=X_Switzerland_test[{'predicted_infections'}]
X_Switzerland_test["Actual infections"] = covid_df_kaggle_Switzerland["Deaths"]
X_Switzerland_test["Date"] = covid_df_kaggle_Switzerland["ObservationDate"]
X_Switzerland_test = X_Switzerland_test.iloc[0:5]
X_Switzerland_test 

,predicted_infections,Actual infections,Date
0,128,98,2020-03-22
1,181,120,2020-03-23
2,238,122,2020-03-24
3,324,153,2020-03-25
4,392,191,2020-03-26
